In [24]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

In [29]:
def Kinematic_model(dt, L, yaw):
    # Linear approximation of the kinematic model
    A = np.array([[1, 0, dt * np.cos(yaw), 0],
                  [0, 1, dt * np.sin(yaw), 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])

    B = np.array([[0, 0],
                  [0, 0],
                  [dt, 0],
                  [0, dt / L]])

    return A, B

In [38]:
def MPC(z_ref, z_initial, Hp=6, ts=0.1, L=2.843, a_min=-1.5, a_max=1.0, 
        delta_f_max=0.9, a_rate_max=1.5, steer_rate=0.175):
    n_states = 4
    n_controls = 2

    # Define CVXPY variables and parameters
    z = cp.Variable((n_states, Hp))
    u = cp.Variable((n_controls, Hp-1))

    # Cost matrices
    Q = np.diag([3.5, 3.5, 25, 80])
    R = np.eye(n_controls)
    R_ = np.eye(n_controls)

    # Objective and constraints
    cost = 0
    constraints = [z[:,0] == z_initial]

    for i in range(Hp-1):
        # Update cost
        cost += cp.quad_form(z_ref[:,i] - z[:,i], Q)
        cost += cp.quad_form(u[:,i], R_)
        if i > 0:
            cost += cp.quad_form(u[:,i] - u[:,i-1], R)

        # Update dynamics and constraints
        A, B = Kinematic_model(ts, L, z_ref[2, i])
        constraints += [z[:,i+1] == A @ z[:,i] + B @ u[:,i]]

        # Apply control and rate constraints
        constraints += [a_min <= u[1, i], u[1, i] <= a_max]
        constraints += [-delta_f_max <= u[0, i], u[0, i] <= delta_f_max]
        if i > 0:
            constraints += [cp.abs(u[1, i] - u[1, i-1]) / ts <= a_rate_max]
            constraints += [cp.abs(u[0, i] - u[0, i-1]) / ts <= steer_rate]

    # Quad program
    qp = cp.Problem(cp.Minimize(cost), constraints)
    qp.solve(solver=cp.ECOS_BB, verbose=False)

    if qp.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        a = np.array(u.value[1, :]).flatten()
        delta = np.array(u.value[0, :]).flatten()
    else:
        a, delta = None, None

    return a, delta

In [ ]:
ts = 0.

In [33]:
def get_ref_traj(type):
    """ 3 types of trajectories available
    A == A right turn with slow speed
    B == A sinusoidal path with morderate speed
    C == CPG winding track"""
    if type == "A":
        T = 81     #total simulation time
        N_steps = int(T/ts)
        v = np.array([0.0]*N_steps)
        # constructing the reference velocity profile
        #for 5s speed is 0
        n = int(5/ts)
        v[:n] = 0
        #for next 45s speed is 3m/s
        n1 = int(45/ts + n)
        v[n:n1] = 3
        #for next 4s speed gradualy reduces to zero
        n = int(4/ts)
        for i in range(n):
            v[n1+i] = v[n1+i-1]-0.75*ts
        # for next 4 sec speed is 0
        n1 = int(n1 + n)
        n  = int(3/ts +n1)
        v[n1:n] = 0.0
        # for next 12s speed is 6m/s
        n1 = int(12/ts + n)
        v[n:n1] = 6
        #for next 10 s the speed delecerates to 0
        n = int(10/ts)
        for i in range(n):
            v[n1+i] = v[n1+i-1]-0.6*ts
        #for next 1s speed is zero
        n1 = int(n1 + n)
        n  = int(1/ts)
        v[n1:n] = 0.0
        # #hence total simulation time is 5+45+4+4+12+10+1 = 81s

        # now based on the refrence velocity we get the x,v,yaw of the vehicle
        x   = np.array([0.0]*N_steps)
        y   = np.array([0.0]*N_steps)
        yaw = np.array([0.0]*N_steps)

        #first 30 secs straight road in x direction
        for i in range(int(30/ts)-1):
            yaw[i+1] = yaw[i]
            x[i+1] = x[i] + v[i]*np.cos(yaw[i])*ts
            y[i+1] = y[i] + v[i]*np.sin(yaw[i])*ts 
        # turining right in 10s 
        for i in range(int(30/ts)-1,int(40/ts)-1):
            yaw[i+1] = yaw[i] + np.pi/20*ts
            x[i+1] = x[i] + v[i]*np.cos(yaw[i])*ts
            y[i+1] = y[i] + v[i]*np.sin(yaw[i])*ts
        #again straight for next 31 secs in y direction
        for i in range(int(40/ts)-1,int(81/ts)-1):
            yaw[i+1] = yaw[i]
            x[i+1] = x[i] + v[i]*np.cos(yaw[i])*ts
            y[i+1] = y[i] + v[i]*np.sin(yaw[i])*ts
    return np.array([x,y,yaw,v])

In [34]:
ref = get_ref_traj('A')

In [37]:
ref

array([[  0.        ,   0.        ,   0.        , ...,  93.94820047,
         93.94820047,  93.94820047],
       [  0.        ,   0.        ,   0.        , ..., 156.79820047,
        156.79820047, 156.79820047],
       [  0.        ,   0.        ,   0.        , ...,   1.57079633,
          1.57079633,   1.57079633],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [40]:
send_prev = 0
prev_accelerations = np.array([0.0] * 6)
prev_deltas = np.array([0.0] * 6)

In [47]:
def solve(x,y,yaw,v):
    """function to call mpc solver"""
    z_intital = np.array([x,y,yaw,v])
    a,delta = MPC(z_ref=ref, z_initial=z_intital)
    if a is None:
        prev_accelerations = prev_accelerations[1:]
        prev_deltas = prev_deltas[1:]
    else:
        prev_accelerations = a
        prev_deltas = delta
    return prev_accelerations[0], prev_deltas[0]



In [53]:

"""runs for 1 trajectory"""
# A,B = Kinematic_model(0,0,0)
# z_initial = A @ np.array([x,y,yaw,v]) + B @ np.array([[0],[0]])
z_initial = [x,y,yaw,v]
states = z_initial
inputs = np.array([[0],[0]])
#step 1
a,delta = solve(z_initial[0],z_initial[1],z_initial[2],z_initial[3])


ValueError: Cannot broadcast dimensions  (4,) (4, 1)

In [50]:
delta

5.574732257305723e-23

In [51]:
for i in range(0,int(81/ts)-1):
    print(i)
    # print(z_ref[2][0]-yaw)
    try:
        z_ref = np.delete(z_ref,0,axis=1)
    except IndexError:
        pass

    A,B = Kinematic_model(yaw[0],delta,ts)
    z = A @ np.array([x,y,yaw,v]) + B @ np.array([[delta],[a]])
    x = z[0]
    y = z[1]
    yaw = z[2]
    v = z[3]
    a,delta = solve(z[0],z[1],z[2],z[3])
    # print(a,delta)
    states = np.append(states,z,axis=-1)
    inputs = np.append(inputs,np.array([[delta],[a]]),axis=-1)

    

0


TypeError: 'int' object is not subscriptable